task1


In [1]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

100%|██████████| 25.7M/25.7M [00:00<00:00, 103MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1


In [6]:
!ls  /root/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1

'IMDB Dataset.csv'


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv("/root/.cache/kagglehub/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1/IMDB Dataset.csv")
# (For speed, you can sample 1000 rows; in a full experiment, use the full dataset)
df = df.sample(1000, random_state=42)

# Encode sentiment: positive -> 1, negative -> 0
df["label"] = df["sentiment"].map({"positive": 1, "negative": 0})

# Retain only 'review' and 'label'
df = df[["review", "label"]]

# Split into training and test sets (you can further split training into validation if desired)
X = df["review"].values
y = df["label"].values
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Training samples:", len(X_train))
print("Validation samples:", len(X_val))
print("Test samples:", len(X_test))

Training samples: 700
Validation samples: 150
Test samples: 150


In [8]:
from transformers import BertTokenizer, BertForSequenceClassification


tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-tiny")
model = BertForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=2)

# Tokenize function with max_length=256
def tokenize_function(texts):
    return tokenizer(list(texts), padding="max_length", truncation=True, max_length=256)

# Tokenize training, validation, and test data
train_encodings = tokenize_function(X_train)
val_encodings   = tokenize_function(X_val)
test_encodings  = tokenize_function(X_test)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

In [9]:
import torch

class IMDBDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Ensure tensors are contiguous
        item = {key: torch.tensor(val[idx]).contiguous() for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx]).contiguous()
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDBDataset(train_encodings, y_train)
val_dataset   = IMDBDataset(val_encodings, y_val)
test_dataset  = IMDBDataset(test_encodings, y_test)

from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

training_args = TrainingArguments(
    output_dir="./results_imdb",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    logging_steps=50,
    report_to=[],
    learning_rate=5e-5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.675293,0.600000,0.675000,0.364865,0.473684
2,0.686900,0.671493,0.593333,0.740741,0.270270,0.396040


Evaluation results: {'eval_loss': 0.6714929342269897, 'eval_accuracy': 0.5933333333333334, 'eval_precision': 0.7407407407407407, 'eval_recall': 0.2702702702702703, 'eval_f1': 0.39603960396039606, 'eval_runtime': 2.3529, 'eval_samples_per_second': 63.752, 'eval_steps_per_second': 4.25, 'epoch': 2.0}


In [10]:
model.save_pretrained("./fine_tuned_imdb_model")
tokenizer.save_pretrained("./fine_tuned_imdb_model")

('./fine_tuned_imdb_model/tokenizer_config.json',
 './fine_tuned_imdb_model/special_tokens_map.json',
 './fine_tuned_imdb_model/vocab.txt',
 './fine_tuned_imdb_model/added_tokens.json')

In [11]:
from huggingface_hub import notebook_login
notebook_login()

In [12]:
model.push_to_hub("ooor/fine_tuned_imdb_model")
tokenizer.push_to_hub("ooor/fine_tuned_imdb_model")

model.safetensors:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ooor/fine_tuned_imdb_model/commit/6dddc807996e0be029df7c586b8c3d950adad3db', commit_message='Upload tokenizer', commit_description='', oid='6dddc807996e0be029df7c586b8c3d950adad3db', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ooor/fine_tuned_imdb_model', endpoint='https://huggingface.co', repo_type='model', repo_id='ooor/fine_tuned_imdb_model'), pr_revision=None, pr_num=None)

In [38]:
from pyngrok import ngrok

ngrok.set_auth_token("2sw2rpZeKdIHeGp5pm5jJi8Sx00_213pk4Rh8GZjkiMMMaHXX")

public_url = ngrok.connect(8000)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://3e63-34-16-247-120.ngrok-free.app" -> "http://localhost:8000"


In [40]:
%%writefile app.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import BertForSequenceClassification, BertTokenizer
from fastapi.middleware.cors import CORSMiddleware
import torch
import numpy as np
import secrets

app = FastAPI()

# For authentication, generate a fixed token for demonstration
API_TOKEN = secrets.token_hex(16)

# Load the fine-tuned custom model and tokenizer from Hugging Face Hub or locally.
custom_model = BertForSequenceClassification.from_pretrained("ooor/fine_tuned_imdb_model")
custom_tokenizer = BertTokenizer.from_pretrained("ooor/fine_tuned_imdb_model")

# For demonstration, we simulate a Llama 3 model integration.
# In a real scenario, you would connect to Groq Cloud API to access Llama 3.
def llama3_predict(text):
    # Define a prompt for Llama 3
    prompt = f"Classify the sentiment of this text as positive or negative: '{text}'"
    # Here, we simply simulate a response.
    # Replace this with actual API calls to Groq Cloud if available.
    return {"sentiment": "positive", "confidence": 0.92}

class AnalyzeRequest(BaseModel):
    text: str
    model: str  # "custom" or "llama"
    token: str  # API token for authentication

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3000"],  #
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/analyze/")
def analyze_text(request: AnalyzeRequest):
    # Verify the token (for demonstration, token check is simple)
    if request.token != API_TOKEN:
        raise HTTPException(status_code=401, detail="Unauthorized: Invalid API token")

    if request.model.lower() == "custom":
        # Use the fine-tuned custom model
        inputs = custom_tokenizer(request.text, return_tensors="pt", padding=True, truncation=True, max_length=256)
        outputs = custom_model(**inputs)
        logits = outputs.logits.detach().numpy()
        confidence = np.max(torch.softmax(torch.tensor(logits), dim=-1).numpy())
        sentiment = "positive" if np.argmax(logits) == 1 else "negative"
        return {"sentiment": sentiment, "confidence": float(confidence)}

    elif request.model.lower() == "llama":
        # Use the Llama 3 model via Groq Cloud API (or simulation)
        return llama3_predict(request.text)

    else:
        raise HTTPException(status_code=400, detail="Invalid model selection.")

@app.get("/token")
def get_api_token():
    return {"api_token": API_TOKEN}

Overwriting app.py


In [33]:
custom_model = BertForSequenceClassification.from_pretrained("ooor/fine_tuned_imdb_model")
custom_tokenizer = BertTokenizer.from_pretrained("ooor/fine_tuned_imdb_model")

In [17]:
!pip install uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00


In [19]:
!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.6 MB/s eta 0:00:00


In [36]:
!pip install pyngrok

In [34]:
import subprocess

process = subprocess.Popen(["python", "-m", "uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"])

print("Uvicorn server started in the background!")

Uvicorn server started in the background!


In [35]:
import requests

#  API token
token_response = requests.get("http://localhost:8000/token")
api_token = token_response.json()["api_token"]
print("API Token:", api_token)

#  /analyze/
payload_custom = {
    "text": "This movie was absolutely fantastic!",
    "model": "custom",
    "token": api_token
}
response_custom = requests.post("http://localhost:8000/analyze/", json=payload_custom)
print("Custom Model Response:", response_custom.json())

#  /analyze/
payload_llama = {
    "text": "This movie was absolutely fantastic!",
    "model": "llama",
    "token": api_token
}
response_llama = requests.post("http://localhost:8000/analyze/", json=payload_llama)
print("Llama Model Response:", response_llama.json())

API Token: 2cb32c208547925c6bae5b09a1d66f88
Custom Model Response: {'sentiment': 'positive', 'confidence': 0.5063790082931519}
Llama Model Response: {'sentiment': 'positive', 'confidence': 0.92}


In [ ]:
prompt = f"Classify the sentiment of this text as positive or negative: '{text}'"